In [1]:
#Eduardo Darrazão
#Rodrigo Faria

In [30]:
import cv2
import time
import numpy as np
import sys
#import matplotlib.pyplot as plt
sys.setrecursionlimit(2000)

global kernel
global kernel2
global ALTURA_MIN
global LARGURA_MIN
global N_PIXELS_MIN
global SOMA
global MEDIA_PIXELS

ALTURA_MIN = 5
LARGURA_MIN = 5
N_PIXELS_MIN = 20
SOMA = 0
kernel = np.ones((5,5),np.uint8)
kernel2 = np.ones((2,2),np.uint8)
MEDIA_PIXELS = 100

In [31]:
class Component:
    def __init__(self, label, n_pixels):
        self.label = label
        self.n_pixels = n_pixels

def rotula (entrada, componentes, largura_min, altura_min, n_pixels_min):
    label = 0.1
    global SOMA
    #Inicia Lista de componentes
    componentes = []
    
    for altura in range(entrada.shape[0]):
        for largura in range(entrada.shape[1]):
            #Se o pixel for branco
            if entrada[altura][largura] == 255:
                #Comece a inundação
                componente = inunda(label, entrada, altura, largura, 1)                
                #Se o objeto inundado tiver mais pixels do que o numero minimo definido
                if componente != 0:
                    qnt = componente.n_pixels/MEDIA_PIXELS
                    while qnt > 0:
                        componentes.append(Component(label, componente.n_pixels/qnt))
                        qnt -= 1
                    
                    #Adicione ao lista de componentes e incremente a label
                    componentes.append(componente)
                    label = label+0.1
    #printando todos os componentes encontrados
    print 'Numero de Componentes: ' + str(len(componentes))
    print 'Soma: ' + str(SOMA)
    print 'Media de pixels: ' + str(SOMA/len(componentes))
    print ''
    SOMA = 0
    return len(componentes)


def inunda(label, entrada, x, y, ind):
    n_pixels_cont = 1
    global SOMA
    entrada[x][y] = label
    
    if (x+1)<entrada.shape[0] or (y+1)<entrada.shape[1]:
        #Visite seu quatro vizinhos e verifique se fazem parte do objeto
        if entrada[x-1][y] == 255:
            #Se fizerem, prossiga com a inundação recursiva nessa direção
            n_pixels_cont = inunda(label, entrada, x-1, y, 0) + 1
        if entrada[x][y-1] == 255:
            n_pixels_cont  = inunda(label, entrada, x, y-1, 0) + 1
        if entrada[x+1][y] == 255:
            n_pixels_cont = inunda(label, entrada, x+1, y, 0) + 1
        if entrada[x][y+1] == 255:
            n_pixels_cont  = inunda(label, entrada, x, y+1, 0) + 1
    
    #A variavel ind verifica se estamos na primeira chamada do objeto, assim, se a inundação foi finalizada
    if ind == 1:
        #Se for, verifique se o numero de pixels do objeto é maior que o minimo definido
        if n_pixels_cont >= N_PIXELS_MIN:
            SOMA += n_pixels_cont
            #Se for, crie um componente e retorne
            componente = Component(label, n_pixels_cont)
            return componente
        else:
            return 0
    #Se não for a primeira chamada, retorne a contagem de pixels do objeto para ser incrementada com as outras chamadas
    else:
        return n_pixels_cont
    

In [32]:
class Component:
    def __init__(self, label, n_pixels):
        self.label = label
        self.n_pixels = n_pixels

def rotula2(entrada, componentes, largura_min, altura_min, n_pixels_min):
    label = 0.1
    global SOMA
    #Inicia Lista de componentes
    componentes = []
    
    for altura in range(entrada.shape[0]):
        for largura in range(entrada.shape[1]):
            #Se o pixel for branco
            if entrada[altura][largura] == 255:
                #Comece a inundação
                componente = inunda(label, entrada, altura, largura, 1)                
                #Se o objeto inundado tiver mais pixels do que o numero minimo definido
                if componente != 0:                    
                    #Adicione ao lista de componentes e incremente a label
                    componentes.append(componente)
                    label = label+0.1
    #printando todos os componentes encontrados
    print 'Numero de Componentes: ' + str(len(componentes))
    print 'Soma: ' + str(SOMA)
    print 'Media de pixels: ' + str(SOMA/len(componentes))
    print ''
    return len(componentes)


def inunda2(label, entrada, x, y, ind):
    n_pixels_cont = 1
    global SOMA
    entrada[x][y] = label
    
    if (x+1)<entrada.shape[0] or (y+1)<entrada.shape[1]:
        #Visite seu quatro vizinhos e verifique se fazem parte do objeto
        if entrada[x-1][y] == 255:
            #Se fizerem, prossiga com a inundação recursiva nessa direção
            n_pixels_cont = inunda(label, entrada, x-1, y, 0) + 1
        if entrada[x][y-1] == 255:
            n_pixels_cont  = inunda(label, entrada, x, y-1, 0) + 1
        if entrada[x+1][y] == 255:
            n_pixels_cont = inunda(label, entrada, x+1, y, 0) + 1
        if entrada[x][y+1] == 255:
            n_pixels_cont  = inunda(label, entrada, x, y+1, 0) + 1
    
    #A variavel ind verifica se estamos na primeira chamada do objeto, assim, se a inundação foi finalizada
    if ind == 1:
        #Se for, verifique se o numero de pixels do objeto é maior que o minimo definido
        if n_pixels_cont >= N_PIXELS_MIN:
            #Se for, crie um componente e retorne
            componente = Component(label, n_pixels_cont)
            return componente
        else:
            return 0
    #Se não for a primeira chamada, retorne a contagem de pixels do objeto para ser incrementada com as outras chamadas
    else:
        return n_pixels_cont
    

In [1]:
def trataImagem(img):
    
    #THRESHOLD ADAPTATIVO
    imgthresh = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 101, -30)        
    
    for img in range(1):
        #OPENING
        imgthresh = cv2.morphologyEx(imgthresh, cv2.MORPH_OPEN, kernel)
    
    #ERODE   
    imgthresh = cv2.erode(imgthresh,kernel2,iterations = 4)
    n_componentes = rotula(imgthresh.copy(), 0, ALTURA_MIN, LARGURA_MIN, N_PIXELS_MIN)
    
    return imgthresh

In [34]:
img1 = cv2.imread("60.bmp", cv2.IMREAD_GRAYSCALE )
img2 = cv2.imread("82.bmp", cv2.IMREAD_GRAYSCALE )
img3 = cv2.imread("114.bmp", cv2.IMREAD_GRAYSCALE )
img4 = cv2.imread("150.bmp", cv2.IMREAD_GRAYSCALE )
img5 = cv2.imread("205.bmp", cv2.IMREAD_GRAYSCALE )

img1thresh = trataImagem(img1)
img2thresh = trataImagem(img2)
img3thresh = trataImagem(img3)
img4thresh = trataImagem(img4)
img5thresh = trataImagem(img5)
    
cv2.imwrite('im1.bmp', img1thresh)
cv2.imwrite('im2.bmp', img2thresh)
cv2.imwrite('im3.bmp', img3thresh)
cv2.imwrite('im4.bmp', img4thresh)
cv2.imwrite('im5.bmp', img5thresh)


Numero de Componentes: 60
Soma: 3130
Media de pixels: 52

Numero de Componentes: 201
Soma: 15879
Media de pixels: 79

Numero de Componentes: 196
Soma: 14641
Media de pixels: 74

Numero de Componentes: 162
Soma: 10956
Media de pixels: 67

Numero de Componentes: 192
Soma: 13532
Media de pixels: 70



True